In [1]:
import numpy as np
import numpy.linalg as nl
import csv

train_files = []
for i in range(1, 13):
    train_files.append('horusrssi/-12dBm0.1secRssi' + str(i) + '.txt')
    
test_file = 'traceRssi-12dB0.1sec1.txt'

In [2]:
def convert_train_data(file_name):
    # File content should be [b_i, rssi]
    #                        ..........
    # Convert it to dictionary dict[index_beacon] = [rss_1, ..., rssi_n]
    
    beacon_dict = {}
    with open(file_name, 'r') as packs:
        packs.readline()
        for pack in packs:
            pack = pack[0: len(pack) - 1]
            space_loc = pack.find(' ')
            beacon = pack[0: space_loc]
            ri = float(pack[space_loc + 1:])

            if beacon not in beacon_dict:
                beacon_dict[beacon] = [ri]
            else:
                beacon_dict[beacon].append(ri)

    return beacon_dict

In [3]:
def get_state_location(file_name = 'standingLocation.csv'):
    # Assumption that metric in csv file is inch, multiply by 0.0254 to get meters.
    
    with open(file_name, 'r') as file:
        file_len = sum(1 for _ in file)
        
    with open(file_name, 'r') as file:
        state_loc = np.zeros((file_len, 2))
        for line in file:
            line = line[0:(len(line)-1)]
            splt = line.split(',')
            state_loc[int(splt[0])-1][0] = float(splt[1])
            state_loc[int(splt[0])-1][1] = float(splt[2])
            
    return 0.0254 * state_loc

get_state_location()

array([[ 1.3716, -0.508 ],
       [ 5.0292, -0.508 ],
       [ 8.6868, -0.508 ],
       [ 1.3716,  1.016 ],
       [ 5.0292,  1.016 ],
       [ 8.6868,  1.016 ],
       [ 1.3716,  2.54  ],
       [ 5.0292,  2.54  ],
       [ 8.6868,  2.54  ],
       [ 4.1148,  4.0894],
       [ 6.858 ,  4.0894],
       [10.5156,  0.4064]])

In [4]:
def convert_test_data(file_name):
    # file header should be [TimeStamp, Beacon, Rssi]
    # convert it to numpy array
    
    with open(file_name, 'r') as file:
        file_len = sum(1 for _ in file)
        
    with open(file_name) as file:
        file.readline() # remove the header
        data = np.zeros((file_len - 1, 3))
        for line, i in zip(file, range(file_len - 1)):
            line = line[0:(len(line)-1)]
            splt = line.split(' ')
            data[i][0] = float(splt[0])
            data[i][1] = int(splt[1])
            data[i][2] = float(splt[2])
            
    return data

#convert_test_data('traceRssi-12dB0.1sec1.txt')       

In [5]:
def get_alpha(beacon_dict, num_beacon = 60):
    # Returned alphas should be [alpha_b1, alpha_b2, ..., alpha_bn]
    # ssvs => signal strength values

    alphas = np.zeros(num_beacon)
    
    for beacon in beacon_dict:
        ssvs = np.array(beacon_dict[beacon])
        num_ssv = len(ssvs)
        
        shift_vec = np.arange(num_ssv - 1) + 1
        ssvs_shift = ssvs[shift_vec]
        ssvs_trunc = ssvs[0: len(ssvs) - 1]
    
        sbar = np.mean(ssvs)
        r1 = abs(np.sum((ssvs_shift - sbar)*(ssvs_trunc - sbar))) / (np.sum((ssvs - sbar) ** 2))
        alphas[int(beacon) - 1] = r1
        
    return alphas
        

In [39]:
def train(state_files, num_beacon = 60): 
    # Get mu and sigma for each state, noted that here it's sigma instead of sigma square.

    num_state = len(state_files)
    state_map = np.zeros((num_state, num_beacon, 2))  # For beacons not in file, mu = -100, sigma = 1/3. ???
    state_map[:,:,0] = -100
    state_map[:,:,1] = 100
    
    for file, si in zip(state_files, range(num_state)): # si represents state_index
        beacon_dict = convert_train_data(file)
        alphas = get_alpha(beacon_dict)   # Alphas => [alpha_b1, alpha_b2, ..., alpha_bn]
      
        for rssis in beacon_dict:  # rssis is beacon number
            alpha = alphas[int(rssis)-1]
            arr = beacon_dict[rssis]
            state_map[si][int(rssis)-1][0] = np.mean(arr)
            state_map[si][int(rssis)-1][1] = np.sqrt(((1 + alpha) / (1 - alpha))*np.var(arr)) 

    return state_map

In [7]:
def process_cluster(cluster, num_beacon = 60):
    # Convert cluster to [beacon_1_avg_rssi, ..., beacon_n_avg_rssi], 0 for beacons not present
    
    ret = np.zeros(num_beacon)
    cnt = np.zeros(num_beacon)
    
    arg = np.argsort(cluster[:,1])
    cluster = cluster[arg]

    for i in range(len(cluster)):        
        ret[int(cluster[i][1])-1] += cluster[i][2]
        cnt[int(cluster[i][1])-1] += 1
        
    cnt[cnt == 0] = 1.0
    
    return ret / cnt

In [8]:
def cluster_test_data(filename, num_beacon = 60):
    # Data should be numpy array
    # Return dictionary with d['i'] = [beacon_1_avg_rssi, ..., beacon_n_avg_rssi], i represents state_i
    
    data = convert_test_data(filename)
    num_rows = data.shape[0]
    max_time = data[num_rows-1][0]
    num_cluster = int(max_time / 10) + 1
    
    start_index = [0]
    d = {}
    
    for i in range(1, num_cluster):
        thred = 10 * i
        index = np.searchsorted(data[:,0], thred)
        start_index.append(index)
        
    for i in range(len(start_index) - 1):
        cluster = data[start_index[i]:start_index[i+1]]
        d[i] = process_cluster(cluster)
    
    #d[len(start_index)-1] = process_cluster(data[start_index[len(start_index)]:])
    
    return d

In [41]:
def test(state_map, test_rssi, state_loc):
    # return state index with max liklihood
    # state_map => (state, beacon, (mu, sigma))
    # state_loc => (state, (x, y))
    # test_rssi => [avg_rssi]

    probs = []
    print(state_map[0][:, 1])
    
    for i in range(state_map.shape[0]):
        map_mu = state_map[i][:, 0]
        map_sigma = state_map[i][:, 1]
        
        
        sum_log = - np.sum(np.square(test_rssi - map_mu) / (2 * map_sigma ** 2)) - \
                  np.sum(np.log(map_sigma))
        
        print(sum_log)
    
        probs.append(1 / np.exp(-sum_log))
        
    probs = np.asarray(probs).reshape(state_map.shape[0], 1)
    probs = probs / np.sum(probs)
 
    res = np.sum(probs * state_loc, axis = 0)
    
    return res

In [20]:
def ss_compensator(prev_state, test_file, state_loc, state_map, thred = 2, d = 0.05):
    # thred => max distance per second
    # prev_state, pred_state => numpy (x, y) in meter metric
    # d => perturbation fraction
    
    pred_state = np.array([])
    min_dist = 0
    final_pert = -1
    test_rssi = process_test(test_file, 2)
    max_index = np.argmax(test_rssi)
    
    for pert in [-0.05, 0, 0.05]:
        pert_arr = np.zeros(len(test_rssi))
        pert_arr[max_index] = pert
       
        cur_pred = test(state_map, test_rssi + pert_arr, state_loc)
        cur_dist = nl.norm(prev_state - cur_pred)
        
        if final_pert == -1 or min_dist > cur_dist:
            final_pert = pert
            pred_state = cur_pred
            min_dist = cur_dist
    
    print("pert: " + str(final_pert))
     
    return pred_state

In [42]:
def main():  
    
    state_loc = get_state_location()
    state_map = train(train_files)
    
    d = cluster_test_data(test_file)
    pred_loc = np.zeros((len(d), 2))
    
    for index in d:
        res = test(state_map, d[index], state_loc)
        pred_loc[index-1][0] = res[0]
        pred_loc[index-1][1] = res[1]
   
main()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()


[  6.90981637  12.86213429   6.12791666   4.18816962  11.44059233
   3.55004088   3.44307794   4.71861063   3.46405623   5.53295682
   4.34025076   5.81099334   3.8461341    3.95128487   1.54456939
   4.57748341   5.49274568   4.85358663   3.83988763   4.28121203
   4.04603172   9.27610334   4.19611368   3.48539615   8.52499582
   8.17988858  10.20752971   5.16837062   6.07846596   4.13350966
   3.81983573   7.39761726   2.80290984   3.57357102   8.20453598
   5.70681799 100.           4.29798607   6.54325647  11.39721232
   1.10415881   6.09975547   5.13099025   4.5989749    4.31093106
   5.70464809   2.04503439   3.26023136   3.44468016   3.98387734
   4.46995503   6.56613377   5.8548843    2.53777243   3.63963864
   6.04137521   2.20389266   4.05754439   4.26452498   5.1946524 ]
-155.08292191476835
nan
nan
-200.38890345258764
-160.13320666427467
-291.8232041870539
-178.98562670106335
-190.9799770998103
-350.7076887660181
-194.701424738185
-179.048779395119
nan
[  6.90981637  12.8621

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:19: RuntimeWarning: overflow encountered in exp


In [43]:
data = convert_test_data(test_file)
cluster_test_data(data)

TypeError: expected str, bytes or os.PathLike object, not numpy.ndarray

In [ ]:
d = {}
d[1] = [1.2]
d['b'] = [1,2]
print(len(d))

In [ ]:
x = np.array([1,2,3,4])
arg = np.array([3,2,1,0])
z[:] = x[arg]
z